# Zillow Housing Data (Initial File Processing)

This code reads in files connected with the Kaggle 2017 Zillow Housing Competition stage 1 where the goal was to find a model that could best predict the z-estimate log error which is essentially the residual error term from Zillow's housing price model.  

The main purpose of this code is to undertake an initial exploration of the data, process the data in chunks, throw out columns that contain insufficient amounts of observations, convert the data into integers or factors where possible and ultimately to output a file CSV file. The file compression takes the dataset from approximately 1.7GBs to around 15MBs.

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

#might need to process in chunks because too big
temp = pd.read_csv("properties_2016.csv", encoding = "ISO-8859-1", nrows=20)
temp.iloc[:,0:20].head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,6037,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,6037,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,6037,NaN,NaN


In [4]:
temp = pd.read_csv("properties_2016.csv", encoding = "ISO-8859-1", usecols=['fips'])
temp['fips'].value_counts()

6037.0    2009362
6059.0     741565
6111.0     222853
Name: fips, dtype: int64

In [5]:
train = pd.read_csv("train_2016.csv", encoding = "ISO-8859-1",parse_dates=["transactiondate"])
print(train.shape)
temp_train = pd.read_csv("train_2016.csv", encoding = "ISO-8859-1", nrows=20)
temp_train.head()



(90811, 3)


,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [6]:
#Examine missing values for all the variables in the list
#get understanding of some of the count values, is it safe to replace them with a 0?

chunk_iter = pd.read_csv("properties_2016.csv", encoding = "ISO-8859-1", chunksize=3000)

cnt = 0
for chunk in chunk_iter:
    if cnt == 0:
        misscounts = chunk.isnull().sum()
        garagecarcntvals = chunk["garagecarcnt"].value_counts()
        poolcntvals = chunk["poolcnt"].value_counts()
        fireplacecntvals = chunk["fireplacecnt"].value_counts()
    else:
        misscounts += chunk.isnull().sum()
        garagecarcntvals += chunk["garagecarcnt"].value_counts()
        poolcntvals += chunk["poolcnt"].value_counts()
        fireplacecntvals += chunk["fireplacecnt"].value_counts()
    cnt += 1

print("Missing Data")
print(misscounts)

Missing Data
parcelid                              0
airconditioningtypeid           2173698
architecturalstyletypeid        2979156
basementsqft                    2983589
bathroomcnt                       11462
bedroomcnt                        11450
buildingclasstypeid             2972588
buildingqualitytypeid           1046729
calculatedbathnbr                128912
decktypeid                      2968121
finishedfloor1squarefeet        2782500
calculatedfinishedsquarefeet      55565
finishedsquarefeet12             276033
finishedsquarefeet13            2977545
finishedsquarefeet15            2794419
finishedsquarefeet50            2782500
finishedsquarefeet6             2963216
fips                              11437
fireplacecnt                    2672580
fullbathcnt                      128912
garagecarcnt                    2101950
garagetotalsqft                 2101950
hashottuborspa                  2916203
heatingorsystemtypeid           1178816
latitude                   

## Missing observations

The number of missing observations contained within the train dataset accounts for a significant proportion of the total observations in the dataset.  Things such as architerctural style, basement square footage are missing for the large majority of houses in the database.  The safest bet would be to throw out all of the data where the majority of properties having missing data and focus only on keeping variables where less than 50% of observations are missing.

In [13]:
temp = pd.read_csv("properties_2016.csv", encoding = "ISO-8859-1", nrows=1000000, low_memory=False)
temp.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,1.000000e+06,273310.000000,1975.000000,536.000000,999992.000000,999997.000000,4280.000000,651930.000000,960325.000000,5739.0,...,915.000000,983698.000000,229418.000000,9.852500e+05,9.893440e+05,1000000.000000,9.808670e+05,9.933410e+05,18863.000000,9.783470e+05
mean,1.296211e+07,1.927826,7.185823,644.014925,2.209717,3.087610,3.730841,5.784104,2.300198,66.0,...,277.095082,1964.279903,1.401045,1.717099e+05,4.215594e+05,2014.999532,2.527255e+05,5.385854e+03,13.867359,6.048501e+13
std,2.619029e+06,3.143820,2.356177,507.503315,1.079544,1.275763,0.496289,1.805565,1.002337,0.0,...,369.448200,23.441561,0.543300,4.612234e+05,7.801097e+05,0.034464,4.502451e+05,9.304081e+03,2.238037,4.724418e+11
min,1.071172e+07,1.000000,2.000000,25.000000,0.000000,0.000000,1.000000,1.000000,1.000000,66.0,...,12.000000,1808.000000,1.000000,1.000000e+00,1.000000e+00,2001.000000,1.000000e+00,2.540000e+00,2.000000,6.037101e+13
25%,1.163642e+07,1.000000,7.000000,264.750000,2.000000,2.000000,3.000000,4.000000,2.000000,66.0,...,100.000000,1950.000000,1.000000,7.474925e+04,1.795390e+05,2015.000000,7.461900e+04,2.459830e+03,14.000000,6.037320e+13
50%,1.253713e+07,1.000000,7.000000,529.000000,2.000000,3.000000,4.000000,7.000000,2.000000,66.0,...,168.000000,1963.000000,1.000000,1.226025e+05,3.060090e+05,2015.000000,1.670000e+05,3.991940e+03,14.000000,6.037572e+13
75%,1.409094e+07,1.000000,7.000000,822.250000,3.000000,4.000000,4.000000,7.000000,3.000000,66.0,...,311.000000,1981.000000,2.000000,1.969398e+05,4.885282e+05,2015.000000,3.071720e+05,6.203260e+03,15.000000,6.059042e+13
max,1.632759e+08,13.000000,21.000000,3419.000000,20.000000,20.000000,5.000000,12.000000,20.000000,66.0,...,4144.000000,2015.000000,41.000000,2.514860e+08,2.827860e+08,2015.000000,9.024622e+07,3.458861e+06,98.000000,4.830301e+14


In [14]:
#this function clean some of the data particular to Zillow
def clean_zillow_chunk(chunk,delete=0): 
    #document all columns that are missing with a negative 1
    miss_cols = ["airconditioningtypeid","unitcnt","poolcnt","fireplacecnt","garagecarcnt","yardbuildingsqft17","numberofstories"]
    for col in miss_cols:
        chunk[col].fillna(-1,inplace=True)
    chunk["bathroomcnt"].fillna(chunk.calculatedbathnbr,inplace=True)
    chunk["calculatedfinishedsquarefeet"].fillna(chunk.finishedsquarefeet12,inplace=True)
    if delete == 1:
        del chunk["calculatedbathnbr"]
        del chunk["finishedsquarefeet12"]
    #Can we use tax value to come up with a hypothetical value of square feet?
    return chunk

In [7]:
import pandas as pd
import numpy as np
import math

#dataset is too big so let's just assume that we are going to compress so that it is easier to work with
#the data
#function to check if an item is a percentage
def is_percentage(item):
    if str(item).endswith("%") | str(item).endswith(" MONTHS"):
        try:
            float(item[:-1])
            return True
        except ValueError:
            return False
    else:
        return False

#function to check if an item is a number
def is_number(item):
    try:
        float(item)
        return True
    except ValueError:
        return False

#Function to automate the data compression tasks
def compress_chunk(chunk, has_id, dropcols=0):
    #check to make sure that id is valid otherwise exceptions are thrown in the data and storage is not efficient (occurs at tail of data)
    if has_id == 1:
        id_is_valid = chunk.apply(lambda item: is_number(item[0]), axis=1)
        chunk = chunk.loc[id_is_valid == True]
        #print("Problem w/selection")

    #select string types
    string_cols = chunk.select_dtypes(include=["object"])
    for s in string_cols.columns:
        chunk[s]= chunk[s].fillna("")
        chunk[s] = chunk[s].str.rstrip()
        #check if the column can be mostly considered a percentage or number (threshold 0.95)
        col_is_percentage = chunk.apply(lambda item: is_percentage(item[s]), axis=1)
        col_is_number = chunk.apply(lambda item: is_number(item[s]), axis=1)
        #print("Column is number:", col_is_number.sum())
        #print(len(col_is_number))
        #print(col_is_percentage.sum())
        #print(len(col_is_percentage))
        if col_is_percentage.sum()/len(col_is_percentage) > 0.95:
            #print("Change to float: ", s)
            chunk[s] = chunk[s].str.rstrip("%")
            chunk[s] = chunk[s].str.rstrip(" MONTHS")
            chunk[s] = chunk[s].astype(float)
        elif col_is_number.sum()/len(col_is_number) > 0.95:
            print(s)
            #drop rows that do not conform to the number requirements so errors are not thrown
            chunk = chunk[col_is_number==True]
            chunk[s] = chunk[s].astype(float,errors="ignore")
        else:
            share_unique = len(string_cols[s].unique())/len(string_cols[s])
            #Make sure that these columns are not subject issues due to case values (put everything into upper case)
            chunk[s] = [st.upper() if st is not None else "" for st in chunk[s].astype(str)]
            if share_unique < 0.50:
                #if a small share is unique and the largest value count is the majority then drop this column from data
                cnt_large_value = chunk[s].value_counts()[0]
                if cnt_large_value/len(chunk[s]) >= 0.99 and dropcols == 1:
                    chunk = chunk.drop(s, 1)
                else:
                    #just change to a category column otherwise
                    #print("Change to category: ", s)
                    chunk[s] = chunk[s].astype("category")
                    #print("error here")
    #want to check for float cols that are really integers
    #that is floor(col) = round(col)
    #dataset['deff'] = np.where(dataset['2016-11'] >= dataset['2016-12'], 0,1)
    float_cols = chunk.select_dtypes(include=["float"])
    float_int_cols = []
    for f in float_cols.columns:
        #Also check if there is little variation in the float columns and drop these columns
        if float_cols[f].std() < 0.0001 and dropcols == 1:
            #print("Dropping", f)
            chunk = chunk.drop(f, 1)
        else:
            temp_floor = np.floor(chunk[f]*10)
            temp_round = np.round(chunk[f]*10,0)
            cnt_diff = np.where(temp_floor==temp_round,0,1).sum()
            #if seems like everything is an integer
            if cnt_diff == 0:
                #print("Downcasting", f)
                float_int_cols.append(f)
    for f in float_int_cols:
        chunk[f]=pd.to_numeric(chunk[f],downcast="integer")
    #Drop columns that have too many missing values (greater than 50%)
    missing = chunk.isnull().sum()
    for key,val in missing.items():  #this data has too many exceptions
        if missing[key]/len(chunk) > 0.50 and dropcols == 1:
            chunk = chunk.drop(key, 1)
    return chunk

# Function to determine optimal chunk size based on memory constraint
def optimal_chunk(file, maxmb, encodeval):
    mem = 0
    numrows = 0
    while mem < maxmb:
        numrows += 500
        temp = pd.read_csv(file, nrows=numrows, encoding=encodeval)
        mem = temp.memory_usage(deep=True).sum()/1048576
    return numrows-500

def compress_data(file,maxmb,dropcols,encodeval,parsedatecols,has_id=0):
    memory_footprints_full = []
    memory_footprints = []
    
    #obtain optimal chunk size
    opt_chunk = optimal_chunk(file, maxmb, encodeval)
    print("Optimal chunk for {}MB: {}".format(maxmb, opt_chunk))
    
    # Get the memory usage needed prior to compression
    chunk_iter = pd.read_csv(file, chunksize=opt_chunk, encoding=encodeval, parse_dates=parsedatecols)
    numrows = 0
    for chunk in chunk_iter:
        numrows += opt_chunk
        memory_footprints_full.append(chunk.memory_usage(deep=True).sum()/1048576)
    total_memory_usage = sum(memory_footprints_full)
    print("Memory usage (prior to compression): {}".format(round(total_memory_usage,2)))
    print("Number of rows in files: {}".format(numrows))
    
    keep_cols = []
    temp = pd.read_csv(file, nrows=2, parse_dates=parsedatecols)
    print("Number of columns in original file: {}".format(len(temp.columns)))
    for c in temp.columns:
        if c not in dropcols:
            keep_cols.append(c)
    print("Keep columns: ", keep_cols)

    # Create synthetic data that will help us identify crucial columns to keep so we do not have to read into data frame
    temp = pd.read_csv(file, nrows=opt_chunk, usecols=keep_cols, encoding=encodeval, parse_dates=parsedatecols)
    temp = clean_zillow_chunk(temp,delete=0)
    temp = compress_chunk(temp,has_id,dropcols=1)
    print("Number of columns to keep: {}".format(len(temp.columns)))
    print(temp.columns)
    newdata = []
    alldata = []
    # use the optimal chunk size and columns to keep to read in the chunks
    chunk_iter = pd.read_csv(file, usecols=temp.columns, encoding=encodeval, chunksize=opt_chunk, parse_dates=parsedatecols)
    num = 0
    for chunk in chunk_iter:
        num += 1
        if int(num/20)==(num/20):
            print("Processing Chunk", num)
        chunk = clean_zillow_chunk(chunk,delete=1)
        chunk = compress_chunk(chunk,has_id,dropcols=0)
        #we want some of the key data, but only certain columns
        alldatacols = ["parcelid","regionidzip","regionidcounty","yearbuilt","latitude","longitude",'propertylandusetypeid',"bathroomcnt","bedroomcnt","taxvaluedollarcnt","taxamount","assessmentyear"]
        alldata.append(chunk[alldatacols])
        #only keep ones that are in the train dataset (particular to the Zillow assignment)
        chunk = pd.merge(chunk, train, how='inner', on='parcelid')
        memory_footprints.append(chunk.memory_usage(deep=True).sum()/1048576)
        #merge data into new set
        newdata.append(chunk)
        #try:
        #    newdata=pd.concat([newdata, chunk], axis=0)
        #except NameError:
        #    newdata=chunk
    total_memory_usage = sum(memory_footprints)
    print("Memory usage (after compression): {}".format(round(total_memory_usage,2)))
    #print("Memory usage (newdata): {}".format(round(newdata.memory_usage(deep=True).sum()/1048576,2)))
    return [newdata,alldata]

In [8]:
#these columns are highly correlated with parcelid and therfore are redundant
mcols99 = ["fips","rawcensustractandblock","censustractandblock"]
drop_cols = ["propertycountylandusecode","propertyzoningdesc", "structuretaxvaluedollarcnt","landtaxvaluedollarcnt","taxdelinquencyyear","taxdelinquencyflag"]
drop_cols.extend(mcols99)
print(drop_cols)

['propertycountylandusecode', 'propertyzoningdesc', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxdelinquencyyear', 'taxdelinquencyflag', 'fips', 'rawcensustractandblock', 'censustractandblock']


In [9]:
traindata, alldata = compress_data("properties_2016.csv", 3, dropcols = drop_cols, encodeval="ISO-8859-1", parsedatecols=None, has_id=0)

Optimal chunk for 3MB: 5000
Memory usage (prior to compression): 1770.83
Number of rows in files: 2990000
Number of columns in original file: 58
Keep columns:  ['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',

In [10]:
#merge data with train so we know which ones are actually sold
#train data has all of the transactions prior to October 15, 2016 and only some after
#since worried about selection let's just drop those prior to October 15, 2016?
#note we do not need to keep information for households where they are not in the train set
#note memory usage after compression is still 44.34 MB (quite huge)
#Note also need to figure out how to replace some of the "NaN" values

#check to make sure we have just the data that is in the train dataset
numrows = 0
for data in traindata:
    numrows += len(data)
print(numrows)

numrows = 0
for data in alldata:
    numrows += len(data)
print(numrows)

90811
2985217


In [11]:
#let's consider markets of los-angeles, Orange, and Ventura as distinct markets (to be evaluated separately)?
#lets try to append all of the data together possible since only 11.99mb so create two sets of data
finaltraindata = traindata[0]
for data in traindata[1:len(traindata)]:
    finaltraindata = pd.concat([finaltraindata,data],axis=0)
print(finaltraindata.shape)

finalalldata = alldata[0]
for data in alldata[1:len(alldata)]:
    finalalldata = pd.concat([finalalldata,data],axis=0)
print(finalalldata.shape)

print("Missing Data:  Train Data")
print(finaltraindata.isnull().sum())

print("Missing Data:  All Data")
print(finalalldata.isnull().sum())

#should we drop the data where latitude and longitude are missing?
#how should we treat this data?

(90811, 29)
(2985217, 12)
Missing Data
parcelid                            0
airconditioningtypeid               0
bathroomcnt                       536
bedroomcnt                        536
buildingqualitytypeid           33447
calculatedfinishedsquarefeet     1197
fireplacecnt                        0
fullbathcnt                      1718
garagecarcnt                        0
heatingorsystemtypeid           34731
latitude                          536
longitude                         536
lotsizesquarefeet               10686
poolcnt                             0
propertylandusetypeid             536
regionidcity                     2339
regionidcounty                    536
regionidneighborhood            54799
regionidzip                       571
roomcnt                           536
unitcnt                             0
yardbuildingsqft17                  0
yearbuilt                        1292
numberofstories                     0
taxvaluedollarcnt                 537
assessmenty

In [12]:
#Read-out the data so we can start from here in the future and not lose valuable time trying to process the data again
#fulldata.to_csv(path_or_buf=None, sep=', ', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.')[source]
finaltraindata.to_csv("processed_train_2016.csv", sep=',', encoding="ISO-8859-1")
finalalldata.to_csv("processed_all_2016.csv", sep=',', encoding="ISO-8859-1")